In [ ]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset
# This is for the progress bar.
from tqdm.auto import tqdm
import random

In [ ]:
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm1 = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation((-90,90),interpolation=transforms.InterpolationMode.BILINEAR),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225), inplace=False)
])

test_tfm2 = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
])

In [ ]:
class FoodDataset(Dataset):

    def __init__(self,path,tfm=test_tfm,files = None):
        super(FoodDataset).__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files != None:
            self.files = files
            
        self.transform = tfm
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
            
        return im,label

In [ ]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
GoogLeNet = models.googlenet(weights=False)
GoogLeNet.fc = nn.Linear(GoogLeNet.fc.in_features, 11)
model1 = GoogLeNet.to(device)
ResNet50 = models.resnet50(weights=False, num_classes=11)
model2 = ResNet50.to(device)
VGG16 = models.vgg16(weights=False, num_classes=11)
model3 = VGG16.to(device)
VGG13 = models.vgg13(weights=False, num_classes=11)
model4 = VGG13.to(device)
EfficientNet = models.efficientnet_b2(weights=False, num_classes=11)
EfficientNet.classifier = nn.Sequential(
    nn.Dropout(p=0.25),
    nn.Linear(in_features=1408, out_features=11, bias=True)
)
model5 = EfficientNet.to(device)

# The number of batch size.
batch_size = 64

In [ ]:
test_set1 = FoodDataset("./test", tfm=test_tfm1)
test_loader1 = DataLoader(test_set1, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
test_set2 = FoodDataset("./test", tfm=test_tfm2)
test_loader2 = DataLoader(test_set2, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

In [ ]:
model1.load_state_dict(torch.load("./googlenet_best.ckpt"))
model2.load_state_dict(torch.load("./resnet50_best.ckpt"))
model3.load_state_dict(torch.load("./vgg16_best.ckpt"))
model4.load_state_dict(torch.load("./vgg13_best.ckpt"))
model5.load_state_dict(torch.load("./efficientnet_best.ckpt"))

model1.eval()
model2.eval()
model3.eval()
model4.eval()
model5.eval()

In [ ]:
from scipy.stats import mode

labels1 = []
labels2 = []
labels3 = []
labels4 = []
labels5 = []

with torch.no_grad():
    for data, _ in tqdm(test_loader1):
        pred1 = model1(data.to(device))
        pred2 = model2(data.to(device))
        pred3 = model3(data.to(device))
        pred4 = model4(data.to(device))
        label1 = np.argmax(pred1.cpu().data.numpy(), axis=1).tolist()
        label2 = np.argmax(pred2.cpu().data.numpy(), axis=1).tolist()
        label3 = np.argmax(pred3.cpu().data.numpy(), axis=1).tolist()
        label4 = np.argmax(pred4.cpu().data.numpy(), axis=1).tolist()
        labels1 += label1
        labels2 += label2
        labels3 += label3
        labels4 += label4

    for data, _ in tqdm(test_loader2):
        pred5 = model5(data.to(device))
        label5 = np.argmax(pred5.cpu().data.numpy(), axis=1).tolist()
        labels5 += label5
    
labels_stack = np.row_stack((labels1, labels2, labels3, labels4, labels5))
prediction, _ = mode(labels_stack, axis=0, keepdims=True)
prediction = prediction.tolist()[0]

In [ ]:
# create test csv
def pad4(i):
    return "0"*(4-len(str(i)))+str(i)
df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(len(test_set1))]
df["Category"] = prediction
df.to_csv("submission.csv",index = False)